In [1]:
import sys
import os
from dotenv import load_dotenv

root_dir = os.path.abspath("..")
sys.path.append(root_dir)
dotenv_path = os.path.join(root_dir, ".env")
load_dotenv(dotenv_path)

True

In [2]:
import polars as pl

In [ ]:
ORIGINAL = os.path.join(root_dir, "kaggle", "train.parquet")

FILE_EMBEDDING = os.path.join(root_dir, "data", "embed_flight_feature_train.parquet")
FILE_FLIGHT = os.path.join(root_dir, "data", "processed_flight_features_train.parquet")
FILE_PRICE = os.path.join(root_dir, "data", "processed_pricing_features_train.parquet")
FILE_USER = os.path.join(root_dir, "data", "processed_user_features_train.parquet")
FILE_ADDITIONAL = os.path.join(root_dir, "data", "processed_additional_features_train.parquet")

### Features

In [4]:
print("[INFO] Reading parquet files...")
df_embed  = pl.read_parquet(FILE_EMBEDDING)
df_flight = pl.read_parquet(FILE_FLIGHT)
df_price  = pl.read_parquet(FILE_PRICE)
df_user   = pl.read_parquet(FILE_USER)
df_add    = pl.read_parquet(FILE_ADDITIONAL)

print("[INFO] Shapes:")
print("embed :", df_embed.shape)
print("flight:", df_flight.shape)
print("price :", df_price.shape)
print("user  :", df_user.shape)
print("add   :", df_add.shape)

[INFO] Reading parquet files...
[INFO] Shapes:
embed : (18145372, 385)
flight: (18145372, 11)
price : (18145372, 10)
user  : (18145372, 10)
add   : (18145372, 8)


In [5]:
if "flight_text" in df_flight.columns:
    df_flight = df_flight.drop("flight_text")
    print("[INFO] Dropped column `flight_text` from flight features -as it has been embedded")

[INFO] Dropped column `flight_text` from flight features -as it has been embedded


In [6]:
assert "row_id" in df_embed.columns
assert "row_id" in df_flight.columns
assert "row_id" in df_price.columns
assert "row_id" in df_user.columns
assert "row_id" in df_add.columns

In [7]:
merged_df = (
    df_embed
    .join(df_flight, on="row_id")
    .join(df_price, on="row_id")
    .join(df_user, on="row_id")
    .join(df_add, on="row_id")
)

In [8]:
merged_df.write_parquet(os.path.join(root_dir, "data", "processed_train_features.parquet"))

### Targets

In [ ]:
d = pl.read_parquet(ORIGINAL)
(
    d
    .select(["selected"])
    .write_parquet(
        os.path.join(root_dir, "data", "processed_train_targets.parquet")
    )
)